# Building a Smile Detector

Using a CNN and OpenCV we will create a real-time smile detector which we can deploy through the webcam.

## Training the smile detector

In [2]:
# import the necessary packages
import keras
from lenet import LeNet as LeNet
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import imutils
import cv2
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

C:\Users\bouta\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import tensorflow as tf
# initialize the list of data and labels
data = []
labels = []
# Smiling Images - read data and label
directory = "SmileCNN-master\\SMILEsmileD-master\\SMILEs\\positives\\positives7"
for imagePath in os.listdir(directory):
    if imagePath.endswith(".jpg"): 
        image = cv2.imread(directory + "\\" + imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = imutils.resize(image, width=28)
        image = tf.keras.preprocessing.image.img_to_array(image)
        data.append(image)
        labels.append("smiling")

# Not Smiling - read data and label
directory = "SmileCNN-master\\SMILEsmileD-master\\SMILEs\\negatives\\negatives7"
for imagePath in os.listdir(directory):
    if imagePath.endswith(".jpg"): 
        image = cv2.imread(directory + "\\" + imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = imutils.resize(image, width=28)
        image = tf.keras.preprocessing.image.img_to_array(image)
        data.append(image)
        labels.append("not_smiling")
        

In [7]:
from sklearn import preprocessing
# Scale the raw pixel intensities to the range [0,1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# convert the labels from integers to vectors
le = preprocessing.LabelEncoder().fit(labels)
labels = tf.keras.utils.to_categorical(le.transform(labels), 2)

In [8]:
# Calcul de classTotals & classWeight
classTotals = labels.sum(axis=0)
classWeight = classTotals.max() / classTotals

In [11]:
print (classTotals)
print (classWeight)

[9475. 3690.]
[1.         2.56775068]


In [12]:
from sklearn import model_selection
# partition the data using 80% for training and 20% for testing
(trainX, testX, trainY, testY) = model_selection.train_test_split(data, labels, test_size=0.20, random_state=1)

In [13]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=1, classes=2)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

[INFO] compiling model...


In [14]:
# compile the model
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=15, verbose=1)

[INFO] training network...
Train on 10532 samples, validate on 2633 samples
Epoch 1/15
10532/10532 [==============================] - 42s 4ms/step - loss: 0.3931 - acc: 0.8221 - val_loss: 0.2643 - val_acc: 0.8899
Epoch 2/15
10532/10532 [==============================] - 43s 4ms/step - loss: 0.2716 - acc: 0.8905 - val_loss: 0.2517 - val_acc: 0.9070
Epoch 3/15
10532/10532 [==============================] - 36s 3ms/step - loss: 0.2478 - acc: 0.9008 - val_loss: 0.2259 - val_acc: 0.9134
Epoch 4/15
10532/10532 [==============================] - 35s 3ms/step - loss: 0.2301 - acc: 0.9097 - val_loss: 0.2297 - val_acc: 0.9104
Epoch 5/15
10532/10532 [==============================] - 35s 3ms/step - loss: 0.2127 - acc: 0.9166 - val_loss: 0.2211 - val_acc: 0.9161
Epoch 6/15
10532/10532 [==============================] - 36s 3ms/step - loss: 0.2117 - acc: 0.9159 - val_loss: 0.2174 - val_acc: 0.9138
Epoch 7/15
10532/10532 [==============================] - 41s 4ms/step - loss: 0.1973 - acc: 0.9255 - 

In [15]:
from sklearn import metrics as mx
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=64)
print(mx.classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=le.classes_))

[INFO] evaluating network...
             precision    recall  f1-score   support

not_smiling       0.95      0.93      0.94      1871
    smiling       0.83      0.89      0.86       762

avg / total       0.92      0.91      0.92      2633



## Running the smile detector in real-time

In [16]:
from keras.models import load_model
import cv2
import imutils
from keras.preprocessing.image import img_to_array
import numpy as np

# load the face detector casscade and smile detector CNN
model = load_model("face_recognition_model.H5")
detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

C:\Users\bouta\Anaconda3\lib\site-packages\keras\engine\saving.py:305: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [17]:
# grab the reference to the webcam
camera = cv2.VideoCapture(0)
print (camera.isOpened ())

True


In [ ]:
while(True):

    
    # grab the current frame
    (grabbed, frame) = camera.read()

    # resize the frame, convert it to grayscale, and then clone the
    # original frame so we can draw on it later in the program
    frame = imutils.resize(frame, width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frameClone = frame.copy()

    rects = detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    
    # loop over the face bounding boxes
    for (fX, fY, fW, fH) in rects:
        # extract the ROI (region of interst) of the face from the grayscale image,
        # resize it to a fixed 28x28 pixels, and then prepare the
        # ROI for classification via the CNN
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (28, 28))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        
        # determine the probabilities of both "smiling" and "not
        # smiling", then set the label accordingly
        (notSmiling, smiling) = model.predict(roi)[0]
        if smiling > notSmiling:
            label = "Smiling" 
        else:
            label = "Not Smiling"

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frameClone, label, (fX, fY - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
        (0, 0, 255), 2)

        # show our detected faces along with smiling/not smiling labels
        cv2.imshow("Face", frameClone)

        # if the ’q’ key is pressed, stop the loop
        if cv2.waitKey(1) or 0xFF == ord('q'):
            break

# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()